In [ ]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install sentencepiece


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install sacremoses


In [ ]:
import pandas as pd
import random
from collections import Counter
import re

# Step 1: Read the dataset from CSV
csv_file = "intent_category.csv"  # Replace with your actual file path
df = pd.read_csv(csv_file)

# Extract columns
instructions = df['instruction'].tolist()
intents = df['intent'].tolist()
categories = df['category'].tolist()

# Step 2: Define placeholders
placeholders = [
    "{{account type}}",
    "{{account category}}",
    "{{refund amount}}",
    "{{currency symbol}}",
    "{{invoice number}}",
    "{{person name}}"
]

# Step 3: Target distribution
target_count = 1500  # Each placeholder should appear exactly 1,500 times
placeholder_counts = {placeholder: 0 for placeholder in placeholders}

# Step 4: Inject placeholders into sentences (in-place)
for idx, sentence in enumerate(instructions):
    # Skip rows that already contain placeholders
    if any(placeholder in sentence for placeholder in placeholders):
        continue
    
    # Randomly select a placeholder that hasn't reached the target count
    available_placeholders = [ph for ph in placeholders if placeholder_counts[ph] < target_count]
    if not available_placeholders:
        break  # Stop if all placeholders have reached the target count
    
    placeholder = random.choice(available_placeholders)
    
    # Randomly decide where to inject the placeholder
    position = random.choice(["beginning", "middle", "end"])
    words = sentence.split()
    
    if position == "beginning":
        augmented_sentence = f"{placeholder} {sentence}"
    elif position == "middle":
        mid = len(words) // 2
        augmented_sentence = " ".join(words[:mid] + [placeholder] + words[mid:])
    else:  # position == "end"
        augmented_sentence = f"{sentence} {placeholder}"
    
    # Overwrite the original instruction with the augmented version
    instructions[idx] = augmented_sentence
    
    # Update the count for the selected placeholder
    placeholder_counts[placeholder] += 1

# Step 5: Update the DataFrame with augmented instructions
df['instruction'] = instructions

# Step 6: Save the augmented dataset to a new CSV file
output_csv_file = "augmented_dataset.csv"  # Replace with your desired output file path
df.to_csv(output_csv_file, index=False)

# Step 7: Validate the augmented dataset
def extract_placeholders(text):
    return re.findall(r'\{\{.*?\}\}', text)  # Match only valid placeholders

# Detect rows with placeholders
df['has_placeholder'] = df['instruction'].apply(lambda x: bool(extract_placeholders(x)))
rows_with_placeholders = df['has_placeholder'].sum()

# Count occurrences of each placeholder
placeholders_extracted = df[df['has_placeholder']]['instruction'].apply(extract_placeholders).explode()
placeholder_counts_final = Counter(placeholders_extracted)

# Convert to DataFrame
placeholder_df = pd.DataFrame.from_dict(placeholder_counts_final, orient='index', columns=['count'])

# Display final results
print(f"Rows with placeholders: {rows_with_placeholders}")
print("\nFinal Distribution of Placeholders:")
print(placeholder_df.sort_values(by='count', ascending=False))
print("\nTotal rows in the dataset:", len(df))


Rows with placeholders: 10581

Final Distribution of Placeholders:
                      count
{{person name}}        2387
{{account type}}       1786
{{account category}}   1704
{{refund amount}}      1696
{{currency symbol}}    1643
{{invoice number}}     1508

Total rows in the dataset: 13947


# Updated


In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import random
import nltk
from nltk.corpus import wordnet
import string
from tqdm import tqdm  # Import tqdm for progress bar

# Download WordNet if not available
nltk.download("wordnet")

# Load a pre-trained paraphrasing model (e.g., T5)
model_name = "Vamsi/T5_Paraphrase_Paws"  # A popular paraphrasing model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Check for GPU availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the selected device

print("device", device)

def paraphrase_sentence(sentence):
    """Generate a paraphrased version of the input sentence using T5."""
    # Prepare input for the model
    input_text = f"paraphrase: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    
    # Generate paraphrased output
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    
    # Decode the output
    paraphrased_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return paraphrased_sentence

def introduce_typos(sentence, num_typos=1):
    """Introduce random typos by swapping adjacent characters."""
    words = list(sentence)
    for _ in range(num_typos):
        idx = random.randint(0, len(words) - 2)
        words[idx], words[idx + 1] = words[idx + 1], words[idx]
    return "".join(words)

def negation_transformation(sentence):
    """Add negation to confuse the model."""
    words = sentence.split()
    if "not" in words or "no" in words:
        return sentence  # Already negated
    insert_idx = random.randint(0, len(words) - 1)
    words.insert(insert_idx, "not")
    return " ".join(words)

def shuffle_sentence(sentence):
    """Shuffle word order slightly."""
    words = sentence.split()
    random.shuffle(words)
    return " ".join(words)

def augment_sentence(sentence):
    """
    Apply multiple augmentations to the sentence:
    - Sentence-level synonym replacement (paraphrasing).
    - Typo introduction.
    - Negation transformation.
    - Word shuffling.
    """
    # Probabilities for each augmentation
    PROB_PARAPHRASE = 0.4  # 40% chance of paraphrasing
    PROB_TYPO = 0.3       # 30% chance of introducing typos
    PROB_NEGATION = 0.3   # 30% chance of adding negation
    PROB_SHUFFLE = 0.2    # 20% chance of shuffling words

    augmented_sentence = sentence

    # Apply paraphrasing (sentence-level synonym replacement)
    if random.random() < PROB_PARAPHRASE:
        augmented_sentence = paraphrase_sentence(augmented_sentence)

    # Apply typo introduction
    if random.random() < PROB_TYPO:
        augmented_sentence = introduce_typos(augmented_sentence)

    # Apply negation transformation
    if random.random() < PROB_NEGATION:
        augmented_sentence = negation_transformation(augmented_sentence)

    # Apply word shuffling
    if random.random() < PROB_SHUFFLE:
        augmented_sentence = shuffle_sentence(augmented_sentence)

    return augmented_sentence

# Function to process sentences in batches with a progress bar
def augment_sentences_in_batches(sentences, batch_size=32):
    """Augment sentences in batches with a progress bar."""
    augmented_sentences = []
    for i in tqdm(range(0, len(sentences), batch_size), desc="Augmenting Sentences"):
        batch = sentences[i:i + batch_size]
        augmented_batch = [augment_sentence(sent) for sent in batch]
        augmented_sentences.extend(augmented_batch)
    return augmented_sentences

# Load the CSV file
input_csv = "final_data_v1.csv"  # Replace with your CSV file path
output_csv = "augmented_data.csv"  # Output file path

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv)

# Assume the columns are named 'sentence' and 'intent'
if 'instruction' in df.columns and 'intent' in df.columns:
    # Extract sentences and intents
    sentences = df['instruction'].tolist()
    intents = df['intent'].tolist()

    # Augment sentences in batches with a progress bar
    augmented_sentences = augment_sentences_in_batches(sentences, batch_size=32)

    # Create a new DataFrame with augmented data
    augmented_df = pd.DataFrame({'instruction': augmented_sentences, 'intent': intents})

    # Save the augmented data to a new CSV file
    augmented_df.to_csv(output_csv, index=False)

    print(f"Augmented data saved to {output_csv}")
else:
    raise ValueError("The CSV file must contain 'instruction' and 'intent' columns.")


C:\Users\reaga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\reaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\reaga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\reaga\.cache\huggingface\hub\models--Vamsi--T5_Paraphrase_Paws. Caching files will still work but in a degraded version that might require more space on your disk. T

device cpu


Augmenting Sentences:  12%|█▏        | 94/762 [22:52<2:42:32, 14.60s/it]


KeyboardInterrupt: 